In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **What is the data**

In [ ]:
data = pd.read_csv("../input/electionfinance/CandidateSummaryAction1.csv")

In [ ]:
data.head()

In [ ]:
data.shape


# Prepare data for analysis
1. drop all columns with above 90% missing value

In [ ]:
#first, visualize missing values
import missingno as msn
msn.matrix(data)

**Observation** 
1. data contains messy features 
2. 

In [ ]:
len(data)

In [ ]:
#create a function to check all columns with missing data greater than 90% and drop them

def process_missing_data(data, threshold, inplace_value):
    #create a list to hold columns with missing value above threshold
    drop_cols = []
    
    #create a variable to store all columns in the dataframe
    all_cols = data.columns
    
    #calculate all columns with missing values percentage greater than the threshold
    missing_percentage = (data[all_cols].isna().sum()/len(data))*100
    
    #create a dataframe to store all candidate columns and their percentage
    missing_df = pd.DataFrame({"cols":all_cols, "percentage":missing_percentage})
    
    #check for threshold condition
    missing_filtered = missing_df[missing_df['percentage'] >= threshold] 
    drop_cols.append(missing_filtered["cols"].tolist())
    
    #drop candidate columns
    drop_cols = drop_cols[0]
    data.drop(columns=drop_cols, inplace= inplace_value)
    
    return data.shape


In [ ]:
process_missing_data(data=data, threshold=90, inplace_value=True)

# **Explore the data**

**First, we see what offices the candidates are campaigning for**

In [ ]:
data['can_off'].value_counts(normalize=True, sort=True)

**Basically, there are three offices namely**
* H : house of represenatative
* S : senator
* P : presidency

**create three dataframes based on the candidate office**

In [ ]:
#first, convert the net_con column to a float data type and modify data inplace
def converter(data, data_col):
    value = data[data_col].str.replace('$','').str.replace(',','').str.replace('(','-').str.replace(')','').astype('float32')
    data[data_col] = value
    return data.head()

#call the function on the net_con feature
converter(data=data, data_col="net_con")

In [ ]:
H_df = data.loc[data['can_off'] == "H"] 
S_df = data.loc[data['can_off'] == "S"]
P_df = data.loc[data['can_off'] == "P"]

In [ ]:
print(f'The shape of the House of assembly data is {H_df.shape}')
print(f'The shape of the senate data is {S_df.shape}')
print(f'The shape of the presidential data is {P_df.shape}')

# let's start by analyzing the house of representative data

**Since we are interested in the finances of the campaign, lets see how much is spent on campaign in each district in a state**


In [ ]:
Amt_per_sta_ds = H_df.groupby(['can_off_sta', 'can_off_dis'])['net_con'].sum().to_frame(name = "total_dis_sum").reset_index()

In [ ]:
Amt_per_sta_ds.head()

In [ ]:
#visualize the state with high spending

#sns.set_theme(style="whitegrid")

plt.figure(figsize=(20,10))

ax = sns.barplot(x="can_off_sta", y="total_dis_sum", data=Amt_per_sta_ds)

We can clearly see that the highest spending state in terms of house of reps election is the MT, let's now go further to see the amount of competitors in that district

In [ ]:
mt_comp = H_df.loc[H_df['can_off_sta'] == 'MT']
mt_comp.head()

In [ ]:
ax = sns.barplot(x='can_nam', y='net_con', data=mt_comp)

we can clearly see that for state MT with just one district, the higest spender won the vote, however, before we conclude, let's check how long each candidate campaigned for

In [ ]:
#to achieve this aim, we make use of the two date columns (cov_sta_dat, cov_end_dat)

mt_comp['cov_sta_dat'] = pd.to_datetime(mt_comp['cov_sta_dat'])
mt_comp['cov_end_dat'] = pd.to_datetime(mt_comp['cov_end_dat'])


In [ ]:
mt_comp['campaign_duration'] = (mt_comp['cov_end_dat'] - mt_comp['cov_sta_dat']).dt.days

In [ ]:
mt_comp

In [ ]:
ax = sns.barplot(x='can_nam', y='campaign_duration', data=mt_comp)

**Observation**
it is easy to see a trend in between the the money spent and the number of days used to campaign by a candidate

**Next, we seek to find out the number of candidates vying for a sit in a district(competitors)**

In [ ]:
competitors = H_df.groupby(['can_off_sta', 'can_off_dis'])['can_id'].count().to_frame(name = "num_of_comp").reset_index()
#eliminate data points where num_of_comp <= 1
#this means that these positions are unopposed
competitors = competitors[competitors['num_of_comp'] > 1]

In [ ]:
competitors.head()

In [ ]:
plt.figure(figsize=(20,10))

ax = sns.barplot(x="can_off_dis", y="num_of_comp", data=competitors)

In [ ]:
#export processed data to csv for analysis in microsoft BI
data.to_csv("stage_f_data.csv", index=False)